In [ ]:
import os
import shutil
import scipy.io
from tqdm import tqdm

def prepare_imagefolder_format(root_dir):
    # Paths to annotation files
    train_anno_path = os.path.join(root_dir, "devkit", "cars_train_annos.mat")
    test_anno_path = os.path.join(root_dir, "cars_test_annos_withlabels.mat")

    # Input image dirs
    train_img_dir = os.path.join(root_dir, "cars_train")
    test_img_dir = os.path.join(root_dir, "cars_test")

    # Output dirs
    out_train_dir = os.path.join(root_dir, "train")
    out_test_dir = os.path.join(root_dir, "test")

    os.makedirs(out_train_dir, exist_ok=True)
    os.makedirs(out_test_dir, exist_ok=True)

    def organize_images(anno_path, img_dir, output_dir):
        annos = scipy.io.loadmat(anno_path)['annotations'][0]
        for a in tqdm(annos, desc=f"Organizing {output_dir}"):
            fname = str(a['fname'][0])               # ✅ Correct field
            label = int(a['class'][0][0]) - 1       # ✅ Correct field
            label_dir = os.path.join(output_dir, f"{label:03d}")
            os.makedirs(label_dir, exist_ok=True)

            src = os.path.join(img_dir, fname)
            dst = os.path.join(label_dir, fname)

            if os.path.exists(src):
                shutil.copy(src, dst)
            else:
                print(f"⚠️ Missing file: {src}")


    organize_images(train_anno_path, train_img_dir, out_train_dir)
    organize_images(test_anno_path, test_img_dir, out_test_dir)

# Run it
prepare_imagefolder_format("/kaggle/MyCBM/stanford_cars")

print("✅ Dataset has been reorganized to ImageFolder format.")


In [ ]:
import os
import zipfile
import urllib.request

# Thư mục đích
save_path = "/content/my_data"
os.makedirs(save_path, exist_ok=True)

# Tải Tiny ImageNet
url = "http://cs231n.stanford.edu/tiny-imagenet-200.zip"
zip_path = os.path.join(save_path, "tiny-imagenet-200.zip")

print("📥 Đang tải Tiny ImageNet...")
urllib.request.urlretrieve(url, zip_path)

print("📦 Đang giải nén...")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(save_path)

print("✅ Tải và giải nén xong.")


In [ ]:
import shutil

val_dir = os.path.join(save_path, "tiny-imagenet-200", "val")
img_dir = os.path.join(val_dir, "images")
anno_file = os.path.join(val_dir, "val_annotations.txt")

# Đọc nhãn từng ảnh
with open(anno_file, "r") as f:
    lines = f.readlines()
    mapping = {line.split("\t")[0]: line.split("\t")[1] for line in lines}

# Tạo thư mục class và di chuyển ảnh
for img_name, class_name in mapping.items():
    class_dir = os.path.join(val_dir, class_name)
    os.makedirs(class_dir, exist_ok=True)
    shutil.move(os.path.join(img_dir, img_name), os.path.join(class_dir, img_name))

# Xóa folder không cần thiết
shutil.rmtree(img_dir)
os.remove(anno_file)

print("✅ Đã tổ chức lại thư mục val đúng định dạng ImageFolder.")


In [5]:
import torch
args = {
    "device": ["0","1"]
}
device_type = args["device"]
gpus = []
print(device_type)
for device in device_type:
    print(device)
    if device_type == -1:
        device = torch.device("cpu")
    else:
        device = torch.device("cuda:{}".format(device))

    gpus.append(device)

['0', '1']
0
1


In [3]:
print(gpus)

[device(type='cuda', index=6)]


In [9]:
import numpy as np
from sklearn.mixture import GaussianMixture
X = np.array([[1, 2], [1, 4], [1, 0], [10, 2], [10, 4], [10, 0]])
gm = GaussianMixture(n_components=1, random_state=0).fit(X)
gm.means_,gm.weights_,gm.covariances_

(array([[5.5, 2. ]]),
 array([1.]),
 array([[[ 2.02500010e+01, -9.25185854e-17],
         [-9.25185854e-17,  2.66666767e+00]]]))

In [12]:
gm.covariances_[0]

array([[ 2.02500010e+01, -9.25185854e-17],
       [-9.25185854e-17,  2.66666767e+00]])